<a href="https://colab.research.google.com/github/NotAndex/Demo/blob/main/time_series_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0

# 1 Setup


In [ ]:
!pip install yfinance
import yfinance as yf

import numpy as np
import pandas as pd
import datetime

from tensorflow.keras.utils import to_categorical

# 2 Data

In [ ]:
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365*3)
symbol = 'BTC-USD'

cc_data = yf.download(symbol, start=start_date, end=today)
cc_data.reset_index(inplace=True,col_fill=('Date'))

#3 Feature engineering examples

## 3.1 Date dependent features

In [33]:
cc_data['year'] = cc_data['Date'].dt.year 
cc_data['quarter'] = cc_data['Date'].dt.quarter 
cc_data['month'] = cc_data['Date'].dt.month 
cc_data['week'] = cc_data['Date'].dt.isocalendar().week 
cc_data['day'] = cc_data['Date'].dt.day
cc_data['wday'] = cc_data['Date'].dt.dayofweek

cc_data.loc[0:5,:]

,Date,Open,High,Low,Close,Adj Close,Volume,year,quarter,month,week,day,wday
0,2018-06-03,7632.089844,7754.890137,7613.040039,7720.250000,7720.250000,4851760128,2018,2,6,22,3,6
1,2018-06-04,7722.529785,7753.819824,7474.040039,7514.470215,7514.470215,4993169920,2018,2,6,23,4,0
2,2018-06-05,7500.899902,7643.229980,7397.000000,7633.759766,7633.759766,4961739776,2018,2,6,23,5,1
3,2018-06-06,7625.970215,7680.430176,7502.009766,7653.979980,7653.979980,4692259840,2018,2,6,23,6,2
4,2018-06-07,7650.819824,7741.270020,7650.819824,7678.240234,7678.240234,4485799936,2018,2,6,23,7,3
5,2018-06-08,7685.140137,7698.189941,7558.399902,7624.919922,7624.919922,4227579904,2018,2,6,23,8,4


Since there is no predetermined (ordinal) relationship in the relevance of days or months we can encode its representation nominal via One-Hot encoding. Forcing an ordinal relationship via an ordinal encoding and allowing the model to assume a natural ordering between categories may result in poor performance or unexpected results (predictions halfway between categories).

In [34]:
monthCat = pd.DataFrame(to_categorical(cc_data.loc[:,['month']]))
cc_data = pd.concat([cc_data,monthCat.loc[:,1:]],axis=1)

dayCat = pd.DataFrame(to_categorical(cc_data.loc[:,['wday']]),
                          columns=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])
cc_data = pd.concat([cc_data,dayCat],axis=1)

In [35]:
cc_data.loc[0:6,["wday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]]

,wday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# 3.2 Technical features (Indicators **$f_{x}$**)

# 3.3 External Features
e.g Sentinment